#Explaination


1. **資料載入與整合:**

  從六家飲料品牌（如一沐日、八曜、迷客夏等）匯入 CSV 檔，統一標記 brand 欄位後合併成單一 DataFrame。
2. **資料清理與數值轉換:**

  處理 like、comment、share 數值型態與空值填補，確保後續計算與建模穩定。
自訂「熱度分數」以 like + 2×comment + 3×share 計算每則貼文的 popularity_score，將「留言」與「分享」權重提高。
3. **情感分析特徵:**

  使用中文 NLP 工具（SnowNLP）對貼文文字內容計算 sentiment_score（0～1），作為模型輸入之一。
4. **標籤熱門貼文:**

  依 popularity_score 排序，將分數前 25% 貼文標為「熱門」（is_popular=1），其餘為非熱門。
5. **多模型訓練與比較:**
  除了 Random Forest，亦訓練並比較 Logistic Regression、SVM、Gradient Boosting、AdaBoost，以及 XGBoost，並蒐集 Accuracy、Precision、Recall、F1-score 等指標。
6. **模型效能視覺化:**

  以熱圖（heatmap）呈現各演算法在上述指標上的表現，快速辨識最佳方案。
7. **品牌情感趨勢分析:**

  計算每日各品牌平均情感分數，繪製折線圖檢視品牌聲量與口碑變化趨勢。

＃補充說明：
1. 熱度分數的由來：

    設計思路：我們希望整合「按讚」、「留言」、「分享」這三項互動行為，一方面反映不同的參與深度，另一方面簡化為可直接比大小的數值指標。

    權重分配原則：
    
    1. 按讚(like)：屬於最輕度互動，只要滑動並點擊即可，因此設置權重為1。
    2. 留言(comment)：需要花費使用者輸入文字的時間，代表中度參與，因此權重為2。
    3. 分享(share)：按一次就能把貼文轉發給別人，，代表最高推廣意願，因此權重設為3。
    
  這樣的加權能夠凸顯「留言」與「分享」對貼文影響力的加乘效果，讓我們能更精準判斷哪些貼文在使用者眼中更具價值或更值得轉發。


2. 為何挑選為何挑選那五個演算法？
    1. **邏輯迴歸(Logistic Regression)**

    性質：線性模型，易於借勢；作為基準模型，常用於二分類問題。

    優點：參數少、運算快速，能快速檢是各特徵（like, comment, share, sentiment_score)與「熱門」之間的線性關係。
    2. **支持向量機(SVM, RBF核)**

    性質：非線性分類器，透過RBF核可以捕捉特徵空間中的複雜邊界。

    優點：參數少、運算快速，能快速檢是各特徵（like, comment, share, sentiment_score)與「熱門」之間的線性關係。
    3. **梯度提升樹(Gradient Boosting Classifier)**

    性質：已逐步建樹提升方式累加弱分類器，強化預測能力。

    優點：對中小型資料集往往能取得不錯的效果；能自動處理部分特徵非線性互動。

    4. **AdaBoost(Adaptive Boosting)**

    性質：透過反覆調整樣本權重，強化難分樣本

    優點：演算法結構簡單，能在基分類器（如淺層決策樹）基礎上快速迭代；適合；適合嘗試是否樣本權重調整能提升效果。

    5. **XGBoost(eXtreme Gradient Boosting)**

    性質：基於梯度提升樹的進階版本，針對速度與正則化作了最佳化，並支援並行計算。

    優點：在許多公開競賽中表現優異；技能處理缺失值，也能透過l1/l2正則化減少過你和風險；適合對複雜互動特徵進行細緻建模。

從線性、非線性、到樹模型、涵蓋了不同「模型複雜度」與「學習機制」，透過這五種模型比較各自的準確度(Accuracy)、精確度(Precision)、召回率(Recall)、F1分數(f1-score)、找到最適合我們貼文特徵的演算法。

3. 各個演算法的超參數有沒有調整？為什麼？

  3.1 基本作法：先用預設參數作為基準
	  •	在最初的探索階段，我們讓上面五種模型先以「預設超參數」進行訓練與交叉驗證，目的在於：
	  1.	快速取得各模型大致效能，確認哪一類心模型最有潛力。
	  2.	判斷是否需要進行更進一步的超參數調校（Hyperparameter Tuning）。
    •	例如：
    •	Logistic Regression 預設 C=1.0、penalty='l2'，
  	•	XGBoost 預設 max_depth=6、learning_rate=0.3、n_estimators=100 等。

  3.2 發現模型間差異後再做調校
	  •	如果某個演算法在基準測試（如 5-fold CV）已經明顯落後，通常可先排除，將資源集中在幾個候選模型上。
	  •	以 XGBoost 為例，我們在預設參數情況下，若其 F1 或 ROC-AUC 已接近專案需求（例如 > 0.85），則進一步使用 GridSearchCV 或 RandomizedSearchCV 優化：
	  •	例如 max_depth（樹的深度） 值測試 [3, 5, 7]；
	  •	learning_rate（學習率） 測試 [0.01, 0.1, 0.2]；
	  •	subsample（抽樣比例） 測試 [0.6, 0.8, 1.0]；
	  •	colsample_bytree（每棵樹的特徵抽樣比例） 測試 [0.6, 0.8, 1.0]。
	  •	而在****SVM 與 Logistic Regression，若輸出結果差距不大，也可考慮：
  	•	SVM：使用 C [0.1, 1, 10]、gamma [‘scale’, 0.1, 1] 調參。
  	•	Logistic：使用 C [0.01, 0.1, 1, 10]、penalty [‘l1’, ‘l2’]（搭配 solver=‘liblinear’）。

  3.3 為何不一開始就大範圍調參？
	  •	若資料量較大，完整調參（尤其是 XGBoost）需花費不少計算資源與時間。
	  •	先以「預設參數」快速檢驗各演算法的「可行性」。
	  •	盲目調整太多參數，反而可能導致在交叉驗證中過度擬合，在真實生產環境反而表現下降。

#Setups

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
pip install snownlp

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from snownlp import SnowNLP
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

In [ ]:
brand_files = {
    "Aniceholiday" : "/content/drive/MyDrive/Machine Learning/finish/一沐日_processed.csv",
    "8yotea" : "/content/drive/MyDrive/Machine Learning/finish/八曜_processed.csv",
    "Naptea" : "/content/drive/MyDrive/Machine Learning/finish/再睡五分鐘_processed.csv",
    "Milksha" : "/content/drive/MyDrive/Machine Learning/finish/迷客夏_processed.csv",
    "Macu" : "/content/drive/MyDrive/Machine Learning/finish/麻古_processed.csv"
}

dfs = []
for brand, file_path in brand_files.items():
  df = pd.read_csv(file_path)
  df['brand'] = brand
  dfs.append(df)

combined_data = pd.concat(dfs, ignore_index=True)

In [ ]:
# Convert like, comment, share to numeric and fill NaN with 0
for col in ['like', 'comment', 'share']:
    combined_data[col] = pd.to_numeric(combined_data[col], errors='coerce').fillna(0)

#Popularity Score Calculate

In [ ]:
# Define popularity score formula
combined_data['popularity_score'] = (
    combined_data['like'] +
    2 * combined_data['comment'] +
    3 * combined_data['share']
)

In [ ]:
# Display top 10 posts with their popularity score
print(combined_data[['brand', 'content', 'like', 'comment', 'share', 'popularity_score']].head(10))

#Sentimant Score using SnowNLP

In [ ]:
# Ensure content is a string, then compute sentiment score
combined_data['sentiment_score'] = combined_data['content'].astype(str).apply(lambda x: SnowNLP(x).sentiments)

In [ ]:
combined_data

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=combined_data, x='sentiment_score', y='popularity_score', hue='brand')
plt.title('Sentiment Score vs Popularity Score by Brand')
plt.xlabel('Sentiment Score (0 = Negative, 1 = Positive)')
plt.ylabel('Popularity Score')
plt.grid(True)
plt.show()

In [ ]:
# Define threshold (75th percentile)
threshold = combined_data['popularity_score'].quantile(0.75)

# Create label column
combined_data['is_popular'] = (combined_data['popularity_score'] >= threshold).astype(int)

In [ ]:
# Feature and label selection
features = combined_data[['like', 'comment', 'share', 'sentiment_score']]
labels = combined_data['is_popular']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train a model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
importances = model.feature_importances_
for feature, importance in zip(features.columns, importances):
    print(f"{feature}: {importance:.3f}")

#Different Models

In [ ]:
# 1) Prepare features & labels
features = combined_data[['like', 'comment', 'share', 'sentiment_score']]
labels   = combined_data['is_popular']
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# 2) Define multiple models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(kernel='rbf', probability=True, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
}

# 3) Train, predict, and collect metrics
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    })

results_df = pd.DataFrame(results).set_index("Model")

# 4) Visualize the comparison
plt.figure(figsize=(8, 5))
sns.heatmap(results_df, annot=True, fmt=".2f", cmap="Blues")
plt.title("Model Performance Comparison")
plt.tight_layout()
plt.show()

# 5) Print the DataFrame
results_df

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier

# 1) Prepare your features & labels
X = combined_data[['like', 'comment', 'share', 'sentiment_score']]
y = combined_data['is_popular']

# 2) Instantiate your model
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# 3) Set up Stratified K-Fold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4) Perform cross-validation
scores = cross_val_score(
    model,        # estimator
    X,            # features
    y,            # labels
    cv=kf,        # cross-validator
    scoring='f1'  # or 'accuracy', 'precision', 'recall', 'roc_auc'
)

# 5) Report results
print(f"5-Fold F1 scores: {scores}")
print(f"Mean F1: {scores.mean():.3f}  ±  {scores.std():.3f}")

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

# 1) Re-instantiate your model (unfitted)
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 2) Define metrics
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# 3) Run cross-validation for each metric
scores = {}
for metric in metrics:
    scores[metric] = cross_val_score(
        model,
        X,
        y,
        cv=kf,
        scoring=metric
    )

# 4) Print results
for metric in metrics:
    print(f"{metric.upper()} per fold: {scores[metric]}")